In [1]:
import numpy as np
import tensorflow as tf
import librosa
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Add
from keras.utils import np_utils
from IPython.display import Audio
from scipy.io import wavfile
import pandas as pd
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile
import IPython.display as ipd
import pickle
import math

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls

In [0]:
%cd ./drive/My Drive/assignment-3

IMPORTING PADDING AND RESHAPING DATA

In [0]:
data = open('trainS_abs.pkl', 'rb')
trainS_abs = pickle.load(data)
data1= open('trainX_abs.pkl', 'rb')
trainX_abs = pickle.load(data1)
data2= open('trainN_abs.pkl', 'rb')
trainN_abs = pickle.load(data2)
data3= open('VN_abs.pkl', 'rb')
VN_abs = pickle.load(data3)
data4= open('VX_abs.pkl', 'rb')
VX_abs = pickle.load(data4)
data5= open('VS_abs.pkl', 'rb')
VS_abs = pickle.load(data5)
data6= open('VS_s.pkl', 'rb')
VS_s = pickle.load(data6)
data7= open('VX_s.pkl', 'rb')
VX_s = pickle.load(data7)
data8= open('VN_s.pkl', 'rb')
VN_s = pickle.load(data8)
data9= open('VN_l.pkl', 'rb')
VN_l = pickle.load(data9)
data10= open('VX_l.pkl', 'rb')
VX_l = pickle.load(data10)
data11= open('VS_l.pkl', 'rb')
VS_l = pickle.load(data11)

In [0]:
for k in range(1200):
  trainS_abs[k]=np.pad(trainS_abs[k], [(0, 200-len(trainS_abs[k])), (0, 0)], mode='constant')
  trainN_abs[k]=np.pad(trainN_abs[k], [(0, 200-len(trainN_abs[k])), (0, 0)], mode='constant')
  trainX_abs[k]=np.pad(trainX_abs[k], [(0, 200-len(trainX_abs[k])), (0, 0)], mode='constant')
  VS_abs[k]=np.pad(VS_abs[k], [(0, 200-len(VS_abs[k])), (0, 0)], mode='constant')
  VN_abs[k]=np.pad(VN_abs[k], [(0, 200-len(VN_abs[k])), (0, 0)], mode='constant')
  VX_abs[k]=np.pad(VX_abs[k], [(0, 200-len(VX_abs[k])), (0, 0)], mode='constant')
  VX_l[k]=np.pad(VX_l[k], [(0, 200-len(VX_l[k])), (0, 0)], mode='constant')

In [0]:
def arraycon(matrix):
  matrix=matrix.tolist()
  matrix=np.asarray(matrix)
  return matrix

In [0]:
trainX_abs=arraycon(trainX_abs)
trainS_abs=arraycon(trainS_abs)
trainN_abs=arraycon(trainN_abs)
VX_abs=arraycon(VX_abs)
VS_abs=arraycon(VS_abs)
VN_abs=arraycon(VN_abs)
VX_l=arraycon(VX_l)

In [0]:
trainS_abs=trainS_abs.reshape(1200,200,513)
trainN_abs=trainN_abs.reshape(1200,200,513)
trainX_abs=trainX_abs.reshape(1200,200,513)


In [0]:
def ibm(srce,n):
    M = []
    for i in range(len(srce)):
        m = 1 * (srce[i] > n[i])
        M.append(m)
    return M

CALCULATING IBM Masks

In [0]:
IBM_train=ibm(trainS_abs,trainN_abs)
IBM_val=ibm(VS_abs,VN_abs)

In [0]:
batch=10
size=513
units=128
sr = 16000
tf.reset_default_graph() 
sl = tf.placeholder(tf.int32, None)
inp_x = tf.placeholder(tf.float32, [None, 200, size])
inp_y = tf.placeholder(tf.float32, [None, 200, size])

Defining network

In [0]:
output, states = tf.nn.dynamic_rnn(tf.contrib.rnn.LSTMCell(units, initializer = tf.keras.initializers.glorot_normal()),inp_x, dtype=tf.float32, sequence_length=sl)
denseout = tf.layers.dense(output, 513, kernel_initializer= tf.keras.initializers.glorot_normal())
finalout = tf.sigmoid(denseout)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method inste

In [0]:
prediction=finalout
loss = tf.reduce_mean(tf.losses.mean_squared_error(prediction[:, :513,:], inp_y[:, :513, :]))
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [0]:
sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

In [0]:
epochs = 10
error = np.zeros(epochs)

In [0]:
X_len=np.zeros(1200)
for i in range(1200):
  X_len[i]=513

In [0]:
for epoch in range(epochs):
    for i in range(100):
        start = i
        end = int(start + batch)
        epoch1_y = np.array(IBM_train[start:end])
        epoch1_x = np.array(trainX_abs[start:end])
        seqlen = np.array(X_len[start:end])
        l, _ = sess.run([loss, optimizer], feed_dict = {inp_x: epoch1_x, inp_y: epoch1_y, sl: seqlen})
        error[epoch] += l
    print('Epoch', epoch+1, 'finished of total', epochs,'; loss: ', error[epoch])

Epoch 1 finished of total 10 ; loss:  13.051703087985516
Epoch 2 finished of total 10 ; loss:  8.84346517920494
Epoch 3 finished of total 10 ; loss:  7.841763764619827
Epoch 4 finished of total 10 ; loss:  7.39099645614624
Epoch 5 finished of total 10 ; loss:  7.002991653978825
Epoch 6 finished of total 10 ; loss:  6.700553931295872
Epoch 7 finished of total 10 ; loss:  6.502574793994427
Epoch 8 finished of total 10 ; loss:  6.248752865940332
Epoch 9 finished of total 10 ; loss:  6.02937251329422
Epoch 10 finished of total 10 ; loss:  5.811690215021372


RUNNING ON VALIDATION SET

In [0]:
y_pred=[]
for i in range(1200):
    epoch2_x = np.zeros((1, trainX_abs[i].shape[0], trainX_abs[i].shape[1]))
    epoch2_x[0,:,:] = VX_abs[i]
    seqlen = np.array(X_len[i])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    y_pred.append(sess.run(finalout, feed_dict = {inp_x:epoch2_x, sl :seqlen }))

In [0]:
for i in range(1200):
  y_pred[i] = 1 * (y_pred[i] > 0.5)
y_pred=np.asarray(y_pred)
y_pred=y_pred.reshape(1200,200,513)

In [0]:
Signal=[]

In [0]:
for i in range(1200):
  S=y_pred[i]*VX_l[i]
  S=S.T
  Signal.append(S)

In [0]:
final_val_out=[]

In [0]:
for j in range(1200):
  x = librosa.istft(Signal[j], win_length = 1024, hop_length = 512)
  x = x[:len(VX_s[j])]
  final_val_out.append(x)

In [0]:
x = librosa.istft(Signal[0], win_length = 1024, hop_length = 512)

In [0]:
x=x[:len(VX_s[0])]

SNR VALUE ON VALIDATION

In [0]:
snr_total=[] 

In [0]:
for i in range(1200):
  SNR = 10*math.log10((np.sum(VS_s[i]**2))/(np.sum((VS_s[i] - final_val_out[i])**2)))
  snr_total.append(SNR)

In [0]:
SNR_VAL=sum(snr_total)/1200
SNR_VAL

4.721594267474021

OUTPUTS ON VALIDATION

In [0]:
librosa.output.write_wav('Val1.input.wav',VX_s[0] ,sr)
ipd.Audio('Val1.input.wav')

In [0]:
librosa.output.write_wav('Val1.output.wav', x,sr)
ipd.Audio('Val1.output.wav')


FOR TEST SIGNALS


In [0]:
data12 = open('test_abs.pkl', 'rb')
test_abs = pickle.load(data12)
data13= open('test_l.pkl', 'rb')
test_l = pickle.load(data13)
data14= open('test_s.pkl', 'rb')
test_s = pickle.load(data14)

In [0]:
for k in range(400):
  test_abs[k]=np.pad(test_abs[k], [(0, 200-len(test_abs[k])), (0, 0)], mode='constant')
  test_l[k]=np.pad(test_l[k], [(0, 200-len(test_l[k])), (0, 0)], mode='constant')

In [0]:
test_abs=arraycon(test_abs)

In [0]:
y_pred_test=[]
for i in range(400):
    epoch3_x = np.zeros((1, trainX_abs[i].shape[0], trainX_abs[i].shape[1]))
    epoch3_x[0,:,:] = test_abs[i]
    seqlen = np.array(X_len[i])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
    y_pred_test.append(sess.run(finalout, feed_dict = {inp_x:epoch3_x, sl :seqlen }))

In [0]:
y_pred_test=np.asarray(y_pred_test)
y_pred_test=y_pred_test.reshape(400,200,513)
for i in range(400):
  y_pred_test[i] = 1 * (y_pred_test[i] > 0.5)

In [0]:
Signal_test=[]

In [0]:
for i in range(400):
  St=y_pred[i]*test_l[i]
  St=St.T
  Signal_test.append(St)

In [0]:
y = librosa.istft(Signal_test[0], win_length = 1024, hop_length = 512)
y=y[:len(test_s[0])]

In [0]:
librosa.output.write_wav('test1.input.wav', test_s[0],sr)
ipd.Audio('test1.input.wav')

In [0]:
librosa.output.write_wav('test1.output.wav',y ,sr)
ipd.Audio('test1.output.wav')